In [ ]:
import numpy as np #fails at making graph using fixinggraphs env
import pandas as pd
import trackpy
import os
import h5py
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from datetime import datetime

import mspt.particle_fitting as fit
import mspt.image_processing as img
import mspt.particle_detection as detect
import mspt.trajectory_analysis1 as traj
import mspt.plotting as plot 
os.environ['NUMEXPR_MAX_THREADS'] = '32'

# Specify directory
#directory = os.path.normpath(r'C:\Users\admin\Desktop\MSPT test') # set manually
directory = img.directoryDialog(os.getcwd()) # or via dialog

# Generate list of .mp or .h5 files to process
filepaths_data = img.find_filepaths(directory, extension='mp', exclude=None)
for file in filepaths_data:
    print(file)
    
assert len(filepaths_data) > 0, 'Cannot find any HDF5 files to process in current directory'

# General parameters
batch_mode = True # Load mp file(s) on the fly without pop-up file dialog
frame_range = [] # Restrict analysis to certain frames, e.g. [0, 2000]. To analyze whole movie, leave list empty.
navg = 1 # Frame averaging, applied before background removal

# Background removal
mode = 'continuous_median' # Choose background removal strategy
window_length = 1001 # Set median window length
save_processed_movies = True # Save movies after background removal
parallel = True # Use multiple cores to perform background substraction. Applies only if GPU=False
GPU = False # Use GPU to perform background substraction (requires CUDA and pytorch). Applies only if parallel=False

# Spot detection
thresh = 0.00055 # Threshold paramter to mask candidate spots
DoG_estimates={ 'T' : 0.1423, 's' : 2.1436, 'sigma' : 1.2921 } # Initial guesses for PSF fitting

# Trajectory linking parameters
dmax = 4. # Maximum displacement of particles per frame (in pixels)
max_frames_to_vanish = 0 # Allow to link particles that where missed in these many frames
minimum_trajectory_length = 5 # Only keep particle that exist for at least this many frames

assert len(frame_range)==2 or len(frame_range)==0, 'frame_range is expected to be either of type [] or [int, int]'

for filename in filepaths_data:
    
    # Apply continuous median background removal
    frames, file = img.mp_reader(batch_mode=batch_mode,
                                 file_to_load=filename,
                                 frame_range=frame_range,
                                 mode=mode,
                                 navg=navg,
                                 window_length=window_length,
                                 parallel=parallel, 
                                 GPU=GPU)
    
    # Get name of video
    name = os.path.splitext(os.path.basename(file))[0]
    # Get current timestamp to prevent overwrite data
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Save processed movies to HDF5 file
    if save_processed_movies:
        saving_folder_movie = os.path.splitext(file)[0] + '_{}{}'.format(mode[11:], window_length)
        processed_movie_file = os.path.join(saving_folder_movie,
                                            os.path.split(saving_folder_movie)[1] + '.h5')
        
        if not os.path.exists(saving_folder_movie):
            os.makedirs(saving_folder_movie)
        
        with h5py.File(processed_movie_file, 'w') as fn:
            fn.create_dataset('frames', data=frames)
            fn.create_dataset('window_length', data=window_length)
            if frame_range:
                fn.create_dataset('frame_range', data=frame_range)
            else:
                fn.create_dataset('frame_range', data=[0, frames.shape[0]])
        print('Saved processed movies to {}'.format(processed_movie_file))

        
    # Detect and fit candidate spots
    fitted_particles = fit.particle_fitter(frames,
                                           halfsize=window_length//2,
                                           thresh=thresh,
                                           frame_range=[],
                                           method='trust-ncg',
                                           DoG_estimates=DoG_estimates)     
    
    # Create folder to save processed data
    if save_processed_movies:
        detections_folder = os.path.join(saving_folder_movie,
                                         'thresh{}_fits{}'.format(thresh, timestamp))
    else:
        detections_folder = '{}_{}{}_thresh{}_fits{}'.format(os.path.splitext(file)[0], # Remove file extension
                                                             mode[11:], # Remove 'continuous_'
                                                             window_length,
                                                             thresh,
                                                             timestamp)   
        
    if not os.path.exists(detections_folder):
        os.makedirs(detections_folder)    
        



    # Save particle detections as csv file
    if len(frame_range) == 0:
        csv_name = '{}_all_frames'.format(name)
        detections_file = os.path.join(detections_folder,
                                       csv_name + '.csv')  
    else:
        csv_name = '{}_frames{}-{}'.format(name, frame_range[0], frame_range[1])
        detections_file = os.path.join(detections_folder,
                                       csv_name + '.csv')
    
    fitted_particles.to_csv(detections_file)
    print('Saved trajectory list to {}'.format(detections_file))

    
    # Link trajectories
    linked_trajectories = trackpy.link_df(fitted_particles, search_range=dmax, memory=max_frames_to_vanish)
    linked_trajectories = linked_trajectories.sort_values(by=['particle', 'frame'])
    trajectories_lenfilt = trackpy.filter_stubs(linked_trajectories, minimum_trajectory_length)

    trajectories_folder = os.path.join(detections_folder,
                                       'dmax{}_mem{}_fits{}'.format(dmax, max_frames_to_vanish, timestamp))
    
    if not os.path.exists(trajectories_folder):
        os.makedirs(trajectories_folder)

    trajectories_file = os.path.join(trajectories_folder,
                                     '{}_trajectories.csv'.format(csv_name))

#     # Workaround in Windows if filename is too long
#     if len(trajectories_file) >= 260: #max is 260
#         trajectories_file = '\\\\?\\'+ trajectories_file

    trajectories_lenfilt.to_csv(trajectories_file)
    print('Saved trajectory list to {}'.format(trajectories_file))

Frame 40272: 9 trajectories present.


In [ ]:
#file = os.path.normpath(r'C:\Users\admin\Desktop\MSPT test\001_sample_median1001.h5') # set manually
file = img.fileDialog(os.getcwd()) # or via dialog
assert os.path.isfile(file), 'File "{}" does not exist'.format(file)

# Load processed movies from HDF5 file
with h5py.File(file, 'r') as h5_file:
    frames = np.asarray(h5_file['frames']).copy()
    window_length = np.asarray(h5_file['window_length']).copy()
    frame_range = np.asarray(h5_file['frame_range']).copy()
    
print('Loaded processed movie {}'.format(file))